In [8]:
import pandas as pd
import re

# --- Caricamento file ---
eucs = pd.read_csv("Other/EUCS PDF official file/EUCS_AnnexA_Extraction.csv")
medina = pd.read_csv("Schemes/MEDINA_EUCS_ControlMapping_Clean.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Other/EUCS PDF official file/EUCS_AnnexA_Extraction.csv'

In [ ]:
eucs

,controlId,baseId,description,assurance_level
0,OIS-01.1-1,OIS-01.1,"The CSP shall define, implement, maintain and ...",Basic
1,OIS-01.2-1,OIS-01.2,The ISMS shall be in accordance to ISO/IEC 27001,Substantial
2,OIS-01.3-1,OIS-01.3,The ISMS shall have a valid certification acco...,High
3,OSI-01.4-1,OSI-01.4,The CSP shall document the measures for docume...,Basic
4,OIS-01.5-1,OIS-01.5,The documentation shall include at least:  Sc...,Substantial
...,...,...,...,...
529,PSS-04.1-1,PSS-04.1,The CSP shall ensure the following aspects if ...,Basic
530,PSS-04.2-1,PSS-04.2,The CSP shall ensure the following aspects if ...,Substantial
531,PSS-04.3-1,PSS-04.3,An integrity check shall be performed and auto...,High
532,PSS-05.1-1,PSS-05.1,The CSP shall allow the CSC to specify the loc...,Substantial


In [ ]:
medina

,EUCS Category,EUCS Control (2022),EUCS Text,Code,C5.2020 GERMANY,SecNumCloud FRANCE,ISO 27002,ISO 27017
0,A1 - Organisation of Information Security,OIS-01 - INFORMATION SECURITY MANAGEMENT SYSTEM,NaN,NaN,OIS-01,NaN,5.4\n5.21,NaN
1,NaN,OIS-02 - SEGREGATION OF DUTIES,NaN,OIS-02,OIS-04,6.1\n 6.2,5.3,CLD.6.3.1
2,NaN,OIS-03 - CONTACT WITH AUTHORITIES AND INTERES...,NaN,NaN,OIS-05,6.3\n 6.4,5.5\n5.6,CLD.6.3.1
3,NaN,OIS-04 - INFORMATION SECURITY IN PROJECT MANAG...,NaN,NaN,OIS-05\nOIS-6\nOIS-7,6.5,5.8,CLD.6.3.1
4,A2 - Information Security Policies,ISP-01 - GLOBAL INFORMATION SECURITY POLICY,NaN,NaN,OIS-02,5.2\n 5.1,5.1,NaN
...,...,...,...,...,...,...,...,...
115,A20 - Product Security,PSS-01 - ERROR HANDLING AND LOGGING MECHANISMS,NaN,NaN,PSS-04,NaN,8.15,NaN
116,NaN,PSS-02 - SESSION MANAGEMENT,NaN,NaN,PSS-06,NaN,8.16,NaN
117,NaN,PSS-03 - SOFTWARE DEFINED NETWORKING,NaN,NaN,PSS-10,NaN,8.20\n8.21,CLD.13.1.4
118,NaN,PSS-04 - IMAGES FOR VIRTUAL MACHINES AND CONT...,NaN,PSS-04,PSS-11,NaN,NaN,NaN
